In [7]:
import warnings
warnings.filterwarnings('ignore')

import ast
import math
import pandas as pd
import glob
import numpy as np
import cv2

# from simplification.cutil import simplify_coords

import matplotlib.pyplot as plt
import matplotlib.style as style
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import json
import os
import datetime as dt
from tqdm import tqdm
import pandas as pd
import numpy as np
from glob import glob
import re
import ast
# import cv2
import csv
import time
import ast
import urllib
from PIL import Image, ImageDraw
from tqdm import tqdm
from dask import bag, threaded
import matplotlib
import matplotlib.pyplot as pltc
import numpy as np
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
from dask import bag, threaded
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.nasnet import NASNetMobile
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.applications import MobileNet
from keras.models import Sequential,Model  #Input

from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
# from keras.layers.normalization import BatchNormalization
# from keras.layers.advanced_activations import LeakyReLU
from keras.layers import LeakyReLU
import os

Base_Size = 256
num_csv = 100
num_class = 340
size = 64
steps = 17578
batchsize = 256
epochs = 100




In [11]:
dataset_folder = "../dataset/new/train_simplified"
shuffled_folder = "../dataset/new/shuffled_train_simplified"


# Create CNN model

In [22]:
num_classes = num_class

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(size,size,1),padding='same'))
model.add(MaxPooling2D((2, 2),padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Flatten())
model.add(Dense(128, activation='linear'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(Dense(num_classes, activation='softmax'))

In [8]:
def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

In [24]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(learning_rate=0.01),
                      metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])

model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 64, 64, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │     1,048,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 340)            │        43,860 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,185,236 (4.52 MB)

 Trainable params: 1,185,236 (4.52 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
#Test
def draw_cv2(raw_strokes, size=size, lw=6):
    img = np.zeros((Base_Size, Base_Size))
    for stroke in raw_strokes:
        for i in range(len(stroke[0]) - 1):
            img = cv2.line(img, (stroke[0][i], stroke[1][i]), (stroke[0][i + 1], stroke[1][i + 1]), 255, lw)
    return (cv2.resize(img, (size, size))/ 255.)

In [13]:
def image_generator(size, batchsize, ks, lw=6):
    print("ks ",ks)
    while True:
        
        for k in np.random.permutation(ks):
            filename = os.path.join(shuffled_folder, 'train_file{}.csv'.format(k))
            for df in pd.read_csv(filename, chunksize=batchsize,nrows=45000):
                x = np.zeros((len(df), size, size))
                df['drawing'] = [ast.literal_eval(pts) for pts in df['drawing'].values]   
                df['drawing'] = df['drawing'].apply(draw_cv2)
                x = np.vstack([a for a in df['drawing']]).reshape((len(df),size,size,1))
                y = tf.keras.utils.to_categorical(df.y, num_classes=num_class)
                yield x,y

In [12]:
def df_to_image_array(df, size=size, lw=6):
    df['drawing'] = [ast.literal_eval(pts) for pts in df['drawing'].values]
    x = np.zeros((len(df), size, size))
    
    df['drawing'] = df['drawing'].apply(draw_cv2)
    x = np.vstack([a for a in df['drawing']]).reshape((len(df),size,size,1))
#     x = np.vstack([a for a in df['drawing']])
    return x

In [28]:
num_csv=100

In [18]:
valid_df = pd.read_csv(os.path.join(shuffled_folder, 'train_file{}.csv'.format(num_csv - 1)), nrows=45000)
# print('hi1')
x_valid = df_to_image_array(valid_df, size)
# print('hi2')
y_valid = tf.keras.utils.to_categorical(valid_df.y, num_classes=num_class)
print(x_valid.shape,y_valid.shape)




hi1
hi2
(45000, 64, 64, 1) (45000, 340)


# Train model

In [ ]:
callbacks = [
    ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.3, patience=5,
                      min_delta=0.005, mode='max', verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3),
    ModelCheckpoint('CNN.weights.h5', monitor='val_top_3_accuracy', mode='max', save_best_only=True, save_weights_only=True)
]


train_datagen = image_generator(size=size, batchsize=batchsize, ks=range(num_csv - 1))
clf = model.fit(
    train_datagen,
    steps_per_epoch=steps,
    epochs=1,
    verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks=callbacks
)



tf.keras.models.save_model(model,'./drive/MyDrive/model1.h5')


ks  range(0, 99)


2024-09-03 11:44:15.676595: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 134217728 exceeds 10% of free system memory.
2024-09-03 11:44:21.639963: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 134217728 exceeds 10% of free system memory.


    1/17578 ━━━━━━━━━━━━━━━━━━━━ 38:46:21 8s/step - categorical_accuracy: 0.0039 - categorical_crossentropy: 5.8313 - loss: 5.8313 - top_3_accuracy: 0.0156

2024-09-03 11:44:21.924471: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 134217728 exceeds 10% of free system memory.
2024-09-03 11:44:22.700575: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 134217728 exceeds 10% of free system memory.


    2/17578 ━━━━━━━━━━━━━━━━━━━━ 5:29:48 1s/step - categorical_accuracy: 0.0039 - categorical_crossentropy: 6.1162 - loss: 6.1162 - top_3_accuracy: 0.0146 

2024-09-03 11:44:23.047516: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 134217728 exceeds 10% of free system memory.


17578/17578 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - categorical_accuracy: 0.5350 - categorical_crossentropy: 2.0024 - loss: 2.0024 - top_3_accuracy: 0.7140

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('./drive/MyDrive/model1.h5')
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(learning_rate=0.01),
                      metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])


model.fit(
    train_datagen,
    steps_per_epoch=steps_per_epoch,
    epochs=2,  # Train for 1 more epochs
    initial_epoch=1,  # Start from epoch 1
    verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks=callbacks
)

tf.keras.models.save_model(model,'./drive/MyDrive/model2.h5')


In [21]:
from tensorflow.keras.models import load_model
model = load_model('./model/model3.h5')
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(learning_rate=0.01),
                      metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])



In [16]:
pred_results = []
chunksize = 10000
reader = pd.read_csv('../dataset/test_simplified.csv', chunksize=chunksize)
for chunk in tqdm(reader):
    imgs = df_to_image_array(chunk)
    pred = model.predict(imgs, verbose=1)
    top_3 =  np.argsort(-pred)[:, 0:3]
    pred_results.append(top_3)
print("Finished test predictions...")



0it [00:00, ?it/s]

  1/313 ━━━━━━━━━━━━━━━━━━━━ 49s 157ms/step

2024-09-06 13:56:44.491018: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 163840000 exceeds 10% of free system memory.


313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 49ms/step


1it [00:26, 26.17s/it]

...

313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 63ms/step


11it [07:50, 43.24s/it]

69/69 ━━━━━━━━━━━━━━━━━━━━ 5s 72ms/step


12it [08:00, 40.05s/it]

Finished test predictions...


In [18]:
import pickle 

In [19]:
dic ={}

with open('save_labels.pkl','rb') as f:
    dic = pickle.load(f)

pred_results = np.concatenate(pred_results)
print("Finished data prep...")

preds_df = pd.DataFrame({'first': pred_results[:,0], 'second': pred_results[:,1], 'third': pred_results[:,2]})
preds_df = preds_df.replace(dic)

Finished data prep...


In [22]:
preds_df.head(50)

,first,second,third
0,train,foot,stereo
1,sea turtle,truck,van
2,ocean,snake,castle
3,mountain,tent,church
4,campfire,ladder,fireplace
5,beach,spider,flashlight
6,wine glass,hot air balloon,hourglass
7,speedboat,crocodile,arm
8,fork,bat,axe
9,hourglass,vase,peanut


In [30]:
reader = pd.read_csv('../dataset/test_simplified.csv')
reader['drawing'][0]


'[[[17, 18, 20, 25, 137, 174, 242, 249, 251, 255, 251, 229, 193, 166, 104, 58, 25, 13, 3], [117, 176, 184, 185, 185, 190, 191, 187, 179, 122, 114, 103, 103, 109, 109, 100, 98, 103, 112]], [[64, 39, 25, 24, 37, 73, 78, 88, 91, 91, 84], [117, 117, 134, 155, 177, 180, 176, 160, 148, 129, 127]], [[203, 188, 181, 175, 174, 188, 207, 219, 225, 226, 215], [122, 120, 127, 137, 160, 169, 173, 161, 145, 133, 128]], [[110, 111, 151, 154, 154, 143, 108], [133, 150, 151, 150, 130, 127, 128]], [[0, 7, 18, 20, 28], [0, 10, 59, 80, 100]]]'